# <center> <img src="figs/LogoUFSCar.jpg" alt="Logo UFScar" width="110" align="left"/>  <br/> <center>Universidade Federal de São Carlos (UFSCar)<br/><font size="4"> Departamento de Computação, campus Sorocaba</center></font>
</p>

<font size="4"><center><b>Disciplina: Aprendizado de Máquina</b></center></font>
  
<font size="3"><center>Prof. Dr. Tiago A. Almeida</center></font>

## <center>Projeto Final</center>

**Aluno**: Eduardo Garcia do Nascimento

**RA/CPF**: 22008732800


---
### Carregamento dos dados

Nesta seção é feita a carga dos atributos em um dataframe só, ou seja, os três datasets são lidos e concatenados para que a redução de atributos leve em conta o que existe de melhor em todos eles.


In [1]:
# -*- coding: utf-8 -*-

# Caminho dos arquivos
FILES_DIRECTORY = "data"

import numpy as np 
import pandas as pd 
import os
from scripts import utils

if __name__ == '__main__':
    
                      
    # importa o arquivo e guarda em um dataframe do Pandas
    set1_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set1.csv'), sep=',', low_memory=False)
    set2_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set2.csv'), sep=',', low_memory=False) 
    set3_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'set3.csv'), sep=',', low_memory=False)
    train_dataset = pd.read_csv(os.path.join(FILES_DIRECTORY, 'train.csv'), sep=',')
    test_dataset  = pd.read_csv(os.path.join(FILES_DIRECTORY, 'test.csv'), sep=',')
    
    # Renomeia colunas concatenando o setX antes de fazer o merge para identificá-las posteriormente
    cols = set1_dataset.columns
    for col in cols:
        set1_dataset = set1_dataset.rename(columns={col:'set1_'+col})
        
    cols = set2_dataset.columns
    for col in cols:
        set2_dataset = set2_dataset.rename(columns={col:'set2_'+col})
    
    cols = set3_dataset.columns
    for col in cols:
        set3_dataset = set3_dataset.rename(columns={col:'set3_'+col})

    # Concatena os datasets em somente um dataset único
    frames = [ set2_dataset ]
    input_dataset = pd.concat(frames, axis=1)
    input_dataset['classe'] = np.nan
    input_dataset.loc[train_dataset['Id'].values,'classe'] = train_dataset['Class'].values
    
    samples_mask = (input_dataset.classe!=-1)&(input_dataset.classe!=1)&(input_dataset.classe!=0)
    samples_df = input_dataset[samples_mask].drop('classe', axis=1)
    
    K_df = input_dataset.loc[test_dataset.iloc[:,:].values.T[0]].copy()
    K_df = K_df.drop('classe', axis=1)
    
    print('A base de dados inicial combinada tem %d amostras com %d atributos.' % (input_dataset.shape[0],
                                                                                 input_dataset.shape[1]))
    
        
   
    utils.beep(1, 400)

A base de dados inicial combinada tem 60842 amostras com 110 atributos.


---
### Pré-processamento e seleção de atributos

Nesta seção são feitas limpezas da base de dados como:

* Remoção de atributos sem variância;
* Tratamento de outliers e dados nulos;
* Seleção dos atributos que terão maior valor para o algoritmo de classificação.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import RFECV
from sklearn.svm import SVC

X_df = input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)]

print(X_df.shape)
print('Separando a base em treino e teste')
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X_df.drop('classe', axis=1), 
                                                    X_df[['classe']], 
                                                    test_size=0.2, 
                                                    stratify=X_df[['classe']],
                                                    random_state=0, 
                                                    )

y_df = X_df[['classe']].copy()
X_df = X_df.drop('classe', axis=1)

# Remove os atributos que são constantes e não oferecem nenhum valor aos algoritmos de classificação
variance_mask = VarianceThreshold().fit(X_train_df).get_support()
X_train_df = X_train_df.iloc[:,variance_mask]
X_test_df = X_test_df.iloc[:,variance_mask]
samples_df = samples_df.iloc[:,variance_mask]
variance_mask = np.append(variance_mask,True)
input_dataset = input_dataset.iloc[:,variance_mask]
print('Atributos removidos por baixa variância: %d' % np.sum(~variance_mask))

variance_mask = VarianceThreshold().fit(X_df).get_support()
X_df = X_df.iloc[:,variance_mask]
K_df = K_df.iloc[:,variance_mask]

# Tratamento de outliers e entradas nulas
outliers_df = X_train_df.copy()
#Q1 = outliers_df.quantile(0.25)
#Q3 = outliers_df.quantile(0.75)
#IQR = Q3 - Q1
#outliers_mask = (outliers_df < (Q1 - 1.5 * IQR)) | (outliers_df > (Q3 + 1.5 * IQR))
#print("Número de outliers substituídos por valores nulos:", np.sum(np.sum(outliers_mask)))
#outliers_df[outliers_mask] = np.nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)

#imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
#imp_mean = imp_mean.fit(outliers_df)
X_train_df.loc[:,:] = imp_mean.transform(X_train_df)
X_test_df.loc[:,:] = imp_mean.transform(X_test_df)
samples_df.loc[:,:] = imp_mean.transform(samples_df)


outliers_df = X_df.copy()
#Q1 = outliers_df.quantile(0.25)
#Q3 = outliers_df.quantile(0.75)
#IQR = Q3 - Q1
#outliers_mask = (outliers_df < (Q1 - 1.5 * IQR)) | (outliers_df > (Q3 + 1.5 * IQR))
#print("Número de outliers substituídos por valores nulos:", np.sum(np.sum(outliers_mask)))
#outliers_df[outliers_mask] = np.nan
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp_mean = IterativeImputer(max_iter=10, random_state=0)
imp_mean = imp_mean.fit(outliers_df)

#imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
#imp_mean = imp_mean.fit(outliers_df)
X_df.loc[:,:] = imp_mean.transform(X_df)
K_df.loc[:,:] = imp_mean.transform(K_df)



# Normalização dos dados entre 0 e 1
scaler = MinMaxScaler().fit(X_train_df)
X_train_df.loc[:,:] = scaler.transform(X_train_df)
X_test_df.loc[:,:] = scaler.transform(X_test_df)
samples_df.loc[:,:] = scaler.transform(samples_df)


scaler = MinMaxScaler().fit(X_df)
X_df.loc[:,:] = scaler.transform(X_df)
K_df.loc[:,:] = scaler.transform(K_df)

features_mask = [ False, False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True ,
  True , True , True , True , True , True , True , True , True , True , True ,False  ,
  True , True , True , True , True , True ,False , True , True , True , True , True  ,
  True , True , True , True , True , True ,False , True , True , True , True , True  ,
  False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True ,
  False,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True,  True ,
  True , True , True , True , True , True , True ,False , True , True , True , True  ,
  True , True , True , True , True , True , True , True , True , True , True , True  ,
  True , True , True , True ,False , True ,False , True , True , True , True , True  ,
  True , True , True , True , True , True , True , True , True , True , True , True  ,
  True ,False ,False ,False , True , True ,False , True , True , True , True , True  ,
  True , True , True , True , True , True , True , True , True , True , True , True  ,
  True ,False , True , True , True , True , True , True , True , True , True ,False  ,
  True ,False ,False , True , True ,False , True , True , True ,False , True , True  ,
  True , True , True , True , True , True , True ,False , True , True ,False , True  ,
  True , True , True , True , True , True , True , True , True , True , True , True  ,
  True , True , True , True ,False ,False , True , True , True , True , True , True  ,
  True , True , True , True , True ,False , True , True , True , True , True , True  ,
  True , True ,False , True , True , True , True , True ,False , True , True , True  ,
 False , True , True , True , True , True , True ,False ,False , True , True , True  ,
  True , True , True ,False , True , True , True , True , True , True , True , True  ,
  True ,False ,False ,False ,False ,False ,False ,False , True , True , True , True  ,
  True , True ,False , True , True , True , True , True , True , True , True , True  ,
 False , True , True ,False , True , True , True , True , True , True , True , True  ,
  True ,False , True , True , True , True ,False , True ,False , True , True , True  ,
  True , True , True]


# Seleciona os melhores atritubos para treinametno do algoritmo de classificação
#print('Selecionando melhores features....................................')
#selector = SelectKBest(f_classif, k=89).fit(
#    X_train_df.values,
#    y_train_df.values)
#features_mask = selector.get_support()
#X_train_df = X_train_df.iloc[:,features_mask]
#X_test_df = X_test_df.iloc[:,features_mask]
#samples_df = samples_df.iloc[:,features_mask]
#features_mask = np.append(features_mask, True)
#
#selector = SelectKBest(f_classif, k=89).fit(
#    X_df.values,
#    y_df.values.T[0])
#features_mask = selector.get_support()
#X_df = X_df.iloc[:,features_mask]
#K_df = K_df.iloc[:,features_mask]



#estimator = svm.SVC(kernel="linear")
#selector = RFECV(estimator, step=1, cv=5, verbose=10)
#selector = selector.fit(X_df.values, y_df.values.T[0])
#print(selector.support_)
#print(selector.ranking_)

'''
[False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True False  True  True  True  True  True
  True  True  True  True  True  True False  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False False False  True  True False  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  True  True  True  True False
  True False False  True  True False  True  True  True False  True  True
  True  True  True  True  True  True  True False  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True False False  True  True  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True
  True  True False  True  True  True  True  True False  True  True  True
 False  True  True  True  True  True  True False False  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True False False False False False False False  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
 False  True  True False  True  True  True  True  True  True  True  True
  True False  True  True  True  True False  True False  True  True  True
  True  True  True]
[32 20  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 21
  1  1  1  1  1  1  7  1  1  1  1  1  1  1  1  1  1  1 44  1  1  1  1  1
 31  1  1  1  1  1  1  1  1  1  1  1 18  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1 27  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  2  1 24  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 19 40 34  1  1 36  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1 41  1  1  1  1  1  1  1  1  1 26  1 10 11  1  1  5  1  1  1 39  1  1
  1  1  1  1  1  1  1 30  1  1 17  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  3  4  1  1  1  1  1  1  1  1  1  1  1 38  1  1  1  1  1  1
  1  1 13  1  1  1  1  1 29  1  1  1 43  1  1  1  1  1  1 25 33  1  1  1
  1  1  1 35  1  1  1  1  1  1  1  1  1 23 22  9  8 42 16 15  1  1  1  1
  1  1 14  1  1  1  1  1  1  1  1  1  6  1  1 28  1  1  1  1  1  1  1  1
  1 45  1  1  1  1 37  1 12  1  1  1  1  1  1]
'''


print('\n\n\n\nBreve avaliação das primeiras amostras')
display(X_train_df.head(10))

print('Avaliação do descritivo do dataset que permite ter uma ideia mais realista dos dados')
display(X_train_df.describe())

utils.beep(1, 500)

(4310, 110)
Separando a base em treino e teste
Atributos removidos por baixa variância: 2




Breve avaliação das primeiras amostras


,set2_Id,set2_D0000,set2_D0001,set2_D0002,set2_D0003,set2_D0004,set2_D0005,set2_D0006,set2_D0007,set2_D0008,...,set2_D0096,set2_D0097,set2_D0100,set2_D0101,set2_D0102,set2_D0103,set2_D0104,set2_D0105,set2_D0106,set2_D0107
30929,0.508180,0.003063,0.076923,0.387707,0.003895,0.020230,0.082680,0.060302,0.150754,0.233668,...,0.000500,0.291667,0.020230,0.233668,0.014034,0.005137,2.706067e-04,0.296482,3.342501e-02,0.004307
10429,0.171103,0.005971,0.083916,0.312074,0.000373,0.040708,0.070655,0.078710,0.109677,0.202581,...,0.010380,0.097222,0.040708,0.202581,0.013567,0.002021,1.214453e-04,0.248677,1.509530e-02,0.010948
32114,0.527665,0.007376,0.104895,0.277729,0.009856,0.029853,0.081539,0.129572,0.200627,0.276907,...,0.085062,0.263889,0.029853,0.276907,0.015487,0.012840,1.474312e-04,0.350052,3.346390e-03,0.022254
58526,0.961951,0.001782,0.055944,0.312684,0.000000,0.077906,0.074250,0.150862,0.193966,0.340517,...,0.000000,0.263889,0.077906,0.340517,0.013085,0.001360,1.767200e-22,0.435345,1.850372e-15,0.006281
31504,0.517635,0.012390,0.097902,0.321321,0.000000,0.028434,0.110745,0.042937,0.081518,0.162414,...,0.028141,0.194444,0.028434,0.162414,0.013085,0.016543,3.707365e-04,0.191039,4.428904e-04,0.012383
42433,0.697338,0.000301,0.020979,0.319706,0.062397,0.055231,0.030481,0.250000,0.400000,0.550000,...,0.038621,0.111111,0.055231,0.550000,0.028292,0.000202,3.144645e-01,0.600000,9.503905e-03,0.001795
57237,0.940757,0.001096,0.013986,0.340038,0.006247,0.021416,0.059352,0.055944,0.090909,0.139860,...,0.000640,0.152778,0.021416,0.139860,0.017999,0.000767,1.751032e-05,0.302326,2.377674e-02,0.001436
40967,0.673233,0.000262,0.118881,0.358704,0.000000,0.014870,0.049601,0.028571,0.057143,0.228571,...,0.000720,0.263889,0.014870,0.228571,0.013085,0.000822,1.436238e-05,0.168067,1.415406e-01,0.000179
42371,0.696318,0.000849,0.013986,0.291873,0.003077,0.037646,0.055215,0.108108,0.207207,0.243243,...,0.000080,0.611111,0.037646,0.243243,0.013262,0.000874,1.387204e-05,0.171285,1.282569e-02,0.002154
11415,0.187316,0.000224,0.020979,0.246044,0.000000,0.026406,0.033535,0.200000,0.233333,0.300000,...,0.000040,0.250000,0.026406,0.300000,0.013085,0.000453,4.497224e-06,0.333333,1.968122e-02,0.001077


Avaliação do descritivo do dataset que permite ter uma ideia mais realista dos dados


,set2_Id,set2_D0000,set2_D0001,set2_D0002,set2_D0003,set2_D0004,set2_D0005,set2_D0006,set2_D0007,set2_D0008,...,set2_D0096,set2_D0097,set2_D0100,set2_D0101,set2_D0102,set2_D0103,set2_D0104,set2_D0105,set2_D0106,set2_D0107
count,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,...,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000,3448.000000
mean,0.497861,0.002827,0.042571,0.312018,0.007250,0.058477,0.056323,0.103970,0.155510,0.249683,...,0.028586,0.184074,0.058477,0.249685,0.015400,0.003663,0.028958,0.321748,0.019069,0.005968
std,0.288957,0.017590,0.039183,0.066906,0.032771,0.076887,0.031470,0.079027,0.094205,0.108015,...,0.119870,0.075871,0.076887,0.108015,0.020722,0.018014,0.087440,0.115996,0.041939,0.021970
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250547,0.000409,0.013986,0.274541,0.000291,0.020350,0.044474,0.058944,0.101367,0.185409,...,0.000120,0.125000,0.020350,0.185409,0.013150,0.000767,0.000016,0.250000,0.003104,0.000897
50%,0.491762,0.001562,0.034965,0.305504,0.002237,0.037142,0.058478,0.090909,0.141877,0.237183,...,0.000500,0.180556,0.037142,0.237183,0.013609,0.002221,0.000097,0.310034,0.009653,0.003051
75%,0.749507,0.003134,0.062937,0.341410,0.006402,0.064305,0.066495,0.130435,0.196652,0.301471,...,0.003080,0.236111,0.064305,0.301471,0.014685,0.004205,0.000603,0.384615,0.021101,0.006506
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


---
### Análise exploratória

Nesta seção são exibidas informações do resultado após os dados serem pré-processados e os atributos selecionados. Dentre as ferramentas para análise exploratória que serão utilizados estão:

* Descritivo resumido da base.
* Análises de covariância e correlação.
* Matriz de disperação entre todos os atributos selecionados. 
* Diagramas de violino para visualização dos quartis e outliers como uma variação aos diagramas de caixa.
* Gráfico de dispersão com a dimensionalidade reduzida a somente 2 atributos.

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from scripts import analise_exploratoria

exploratory_analisys = False

if exploratory_analisys == True:
    # Análise do balanceamento das classes
    print('Análise e visualização dos dados:')
    ax = sns.countplot(x='classe', data=pd.concat([X_train_df,y_train_df], axis=1), label="Contagem")

    N,P = y_train_df.classe.value_counts()
    print('Número de posts comuns:', N)
    print('Número de posts phishing:', P)
    plt.show()

    # Análise da matriz scatter pra que entendamos a relação entre os atributos
    print('Análise da matriz de dispersão')
    sns.pairplot(input_dataset.loc[(input_dataset['classe'] == -1) | (input_dataset['classe'] == 1)], hue='classe', height=3.5);
    plt.show()

    # matrizes de covariancia e correlação
    print('Análise das matrizes de covariância ')
    df_covariance = X_train_df.iloc[:,:-1].cov()
    df_correlation = X_train_df.iloc[:,:-1].corr()
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20, 8))
    plt.title('Covariância')
    sns.heatmap(df_covariance, annot=True, xticklabels=df_correlation.columns, 
                yticklabels=df_correlation.columns, ax=ax1)
    plt.title('Correlação')
    sns.heatmap(df_correlation, annot=True, xticklabels=df_correlation.columns, 
                yticklabels=df_correlation.columns, ax=ax2)
    plt.show()


    # Diagramas de caixa
    plt.figure(figsize=(20,10))
    data = pd.melt(pd.concat([X_train_df,y_train_df], axis=1),
                   id_vars="classe", var_name="features", value_name='value')
    sns.boxplot(x='features', y='value', hue='classe', data=data)
    plt.show()

    # Diagramas de violino
    plt.figure(figsize=(20,10))
    sns.violinplot(x='features', y='value', hue='classe', data=data, split=True, inner="quartile")
    plt.show()

    # Separação de atributos e classe para 
    analise_exploratoria.printPCA(X_train_df.values,y_train_df.values.T[0])    

    analise_exploratoria.printJointPlot(X_train_df,y_train_df)    

utils.beep(1, 600)   

---
### Preparo dos dados e experimentos para encontrar os melhores hiperparâmetros

Nesta seção os dados são separados em duas partes: treino e testes. Esta estratégia foi utilizada para posterior comparação com a validação de modelos utilizandos K-folds.
Para encontrar os melhores hiperparâmetros foi utilizada a classe GridSearchCV e devido o seu alto custo computacional e não existência da necessidade de executá-las sempre, a sua chamada é condicionada às variáveis booleanas evaluate_svm_hiperparameters, evaluate_rfc_hiperparameters e evaluate_lrc_hiperparameters serem verdadeiras.

In [4]:
from scripts import preprocessamento
from scripts import experimentos
from scripts import utils

X_train = X_train_df.values
y_train = y_train_df.values.T[0]
X_test = X_test_df.values
y_test = y_test_df.values.T[0]

X = X_df.values
y = y_df.values.T[0]

# Utilize as flags abaixo somente para avaliação dos hiperparâmetros pois elas demoram muito pra serem executadas
evaluate_svm_hiperparameters = False # Busca os melhores parâmetros para as máquinas de vetores de suporte
evaluate_rfc_hiperparameters = False # Busca os melhores parâmetros para as florestas aleatórias
evaluate_knn_hiperparameters = False # Busca os melhores parâmetros para o KNN
evaluate_mlp_hiperparameters = False # Busca os melhores parâmetros para o KNN

print('Removendo amostras outliers - Removido pois estava piorando os resultados')
#X_train, y_train = preprocessamento.remove_outliers(X_train, y_train)
#X,y = preprocessamento.remove_outliers(X, y)

scores = [ 
        #    'balanced_accuracy', 
        #    'f1', 
            'roc_auc'
        ]

if evaluate_svm_hiperparameters == True:
    experimentos.find_best_svm(X,y, scores)
    
if evaluate_rfc_hiperparameters == True:    
    experimentos.find_best_rfc(X, y, scores)
        
if evaluate_knn_hiperparameters == True:
    experimentos.find_best_knn(X, y, scores)

if evaluate_mlp_hiperparameters == True:
    experimentos.find_best_mlp(X, y, scores)
    

utils.beep(1, 700)

Removendo amostras outliers - Removido pois estava piorando os resultados


---
### Experimento

Nesta seção, o experimento deve ser conduzido, utilizando os protocolos experimentais ensinados no curso e executando os métodos inteligentes

In [18]:
# Importa os classificadores usados nesse trabalho
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm

from scripts import experimentos
from scripts import preprocessamento

collect_more_samples = False
collect_more_samples_again = False

X_train_bal, y_train_bal = X_train, y_train

scoring=['roc_auc', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted', 'accuracy', 'balanced_accuracy', 'precision', 'recall']

results = pd.DataFrame()

model_list = [
#    ['SVM poly', svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
#    ['SVM linear', svm.SVC(kernel='linear', C=100, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['SVM rbf', svm.SVC(kernel='rbf', C=1000, gamma=0.0001, class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1), 0],
    ['SVM rbf RA 50', svm.SVC(kernel='rbf', C=50, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
    ['SVM rbf RA 20', svm.SVC(kernel='rbf', C=20, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
    ['SVM rbf RA', svm.SVC(kernel='rbf', C=10, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
    ['SVM Optimal', svm.SVC(kernel='rbf', C=1, gamma='scale', class_weight='balanced', probability=True,random_state=1), 0],
#    ['Random Forest', RandomForestClassifier(max_depth=2, class_weight='balanced', random_state=1), 0],
#    ['Random Forest Optimal', RandomForestClassifier(criterion='gini', max_depth=6, max_features='auto', n_estimators=500, class_weight='balanced', random_state=1), 0],
#    ['Logistic Regression', LogisticRegression(random_state=1, class_weight='balanced', max_iter=15000), 0],
#    ['Multinomial NB', MultinomialNB(), 0],
#    ['MLP', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(600,), random_state=1, max_iter=5000, ), 0],
#    ['MLP7', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(700,), random_state=1, max_iter=5000, ), 0],
#    ['MLP rec3', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(600,40,), random_state=1, max_iter=5000, ), 0],
    #['MLP Optimal', MLPClassifier( alpha=1e-5, hidden_layer_sizes=(600,), random_state=1, max_iter=5000, ), 0],
#    ['KNN', KNeighborsClassifier(n_neighbors=1, leaf_size=4, weights='distance'), 0],  
#    ['KNN Optimal', KNeighborsClassifier(algorithm='auto', n_neighbors=29, leaf_size=1, weights='distance'), 0],
#    ['AdaBoost', AdaBoostClassifier(n_estimators=100, random_state=0), 0],
#    ['GradBoost', GradientBoostingClassifier(random_state=0), 0],
]

# Pré balanceamento dos dados utilizando a técnica de oversampling
X_train_bal, y_train_bal = preprocessamento.oversample(X_train_bal, y_train_bal, times=5)
X_bal, y_bal = preprocessamento.oversample(X, y, times=5)

# População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
if collect_more_samples == True:
    semi_model = svm.SVC(kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1)
    
    X_semi, y_semi, proba_mask = preprocessamento.add_samples(semi_model, X_bal, y_bal, samples_df.values)

    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    
    pos_mask = y_semi==1
    X_semi_positives = X_semi[pos_mask]
    y_semi_positives = y_semi[pos_mask]
    
    X_semi_positives = X_semi_positives
    y_semi_positives = y_semi_positives
    
    X_train_bal = np.concatenate([X_train_bal, X_semi_positives], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi_positives], axis=0)
    X_bal = np.concatenate([X_bal, X_semi_positives], axis=0)
    y_bal = np.concatenate([y_bal, y_semi_positives], axis=0)

if collect_more_samples_again == True:
    X_train_bal, y_train_bal = preprocessamento.oversample(X_train_bal, y_train_bal)
    X_bal, y_bal = preprocessamento.oversample(X_bal, y_bal)

    # População dos dados não classificados com o melhor classificador encontrado com os resultados mais confiáveis
    samples = samples[~proba_mask]
    X_semi, y_semi, proba_mask = preprocessamento.add_samples(semi_model, X_train_bal, y_train_bal, samples)
    print('Número de amostras acrescentadas ao dataset de treinamento:', X_semi.shape[0], 
                                                                          np.sum(y_semi==-1), np.sum(y_semi==1))
    X_train_bal = np.concatenate([X_train_bal, X_semi], axis=0)
    y_train_bal = np.concatenate([y_train_bal, y_semi], axis=0)
    X_bal = np.concatenate([X_bal, X_semi], axis=0)
    y_bal = np.concatenate([y_bal, y_semi], axis=0)

print(X_bal.shape, X_train_bal.shape, np.sum(y_bal==-1),np.sum(y_bal==1), np.sum(y_train_bal==-1),np.sum(y_train_bal==1))

for model in model_list:
    print(model[0] + '------------------------------------------------------------------------------------')
    scores = experimentos.evaluate_model(model[1], X_train_bal, y_train_bal, X_test, y_test)

    for score in scores:
        results.loc[model[0], score] = np.mean(scores[score])
    


utils.beep(1, 800)

Balaceamento antes da SOBREamostragem 3230 218
Balaceamento após da SOBREamostragem 3230 1090
Balaceamento antes da SOBREamostragem 4038 272
Balaceamento após da SOBREamostragem 4038 1360
(5398, 107) (4320, 107) 4038 1360 3230 1090
SVM rbf------------------------------------------------------------------------------------
SVM rbf RA 50------------------------------------------------------------------------------------
SVM rbf RA 20------------------------------------------------------------------------------------
SVM rbf RA------------------------------------------------------------------------------------
SVM Optimal------------------------------------------------------------------------------------


---
### Análise dos Resultados

Nesta seção, os resultados devem ser exibidos e comparados, através de tabelas e gráficos

In [19]:
from scripts import analise_resultados

def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    
    return ['background-color: yellow' if v else '' for v in is_max]

pd.set_option('precision', 4)
results_highlited = results.style.apply(highlight_max)
display(results_highlited)

#best_model.fit(X,y)
#y_pred = best_model.predict(X_test)
#analise_resultados.print_rocauc_curve(y_test, y_pred)
#analise_resultados.plot_decision_boundaries(X_train, y_train, svm.SVC, kernel='poly', class_weight='balanced', decision_function_shape='ovr', probability=True,random_state=1)


utils.beep(3, 1000)

,bal_acc_train,bal_acc_test,f1_weighted,f1_micro,precision,recall,mcc,roc_auc,TP,TN,train_time,predict_time
SVM rbf,0.7345,0.6429,0.8231,0.7680,0.1350,0.5000,0.1641,0.6774,635.0000,27.0000,11.4442,0.3556
SVM rbf RA 50,0.9238,0.6807,0.8698,0.8387,0.1942,0.5000,0.2381,0.7207,696.0000,27.0000,6.6385,0.1803
SVM rbf RA 20,0.8993,0.6875,0.8575,0.8190,0.1812,0.5370,0.2337,0.7293,677.0000,29.0000,6.7552,0.2152
SVM rbf RA,0.8709,0.6837,0.8529,0.8121,0.1747,0.5370,0.2258,0.7403,671.0000,29.0000,7.0703,0.2142
SVM Optimal,0.8061,0.6578,0.8311,0.7796,0.1458,0.5185,0.1838,0.7307,644.0000,28.0000,8.9891,0.2714


---
### Criação do arquivo de sumissão no Kaggle

Na etapa final, o arquivo submission.csv é criado para ser enviado ao Kaggle de acordo com os padrões pré-definidos pela proposta do desafio.

In [20]:
submit_preference_score = 'SVM rbf RA'

for model in model_list:
    if model[0]==submit_preference_score:
        best_model = model[1]
        
# Montagem do dataset de teste para envio para o Kaggle


print('Criando arquivo de submissão para o modelo: ' + submit_preference_score)    
    
print('Imprimindo arquivo submission.csv ...')
clf= best_model.fit(X_bal, y_bal)
y_pred_submission = clf.predict_proba(K_df)[:,1]
result = np.zeros((K_df.shape[0],2))
for i in range(K_df.shape[0]):
    result[i][0] = test_dataset.iloc[:,:].values.T[0][i]
    result[i][1] = y_pred_submission[i]
resultdf = pd.DataFrame(data=result, columns=["Id", "Predicted"])
resultdf['Id'] = resultdf['Id'].astype(int)
resultdf['Predicted'] = resultdf['Predicted'].round(decimals=5)
resultdf.to_csv('submission.csv', index=False, float_format='%.5f')
print('####################################################################################################')

utils.beep(5, 200)

Criando arquivo de submissão para o modelo: SVM rbf RA
Imprimindo arquivo submission.csv ...
####################################################################################################
